# 汇率预测机器人

In [2]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

class CurrencyLSTMModel:
    def __init__(self, currency_name, data_path, look_back=1):
        self.currency_name = currency_name
        self.data_path = data_path
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_and_prepare_data(self):
     dataset = read_csv(self.data_path, header=0, index_col=0)
    # 直接用全部数据（只有汇率一列）
     values = dataset.values.astype('float32')
     scaled = self.scaler.fit_transform(values)
     reframed = series_to_supervised(scaled, self.look_back, 1)
     values = reframed.values
     n_train = int(len(values) * 0.7)
     train = values[:n_train, :]
     test = values[n_train:, :]
     train_X, train_y = train[:, :-1], train[:, -1]
     test_X, test_y = test[:, :-1], test[:, -1]
     train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],1))
     test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],1))
     return train_X, train_y, test_X, test_y

    def build_model(self, input_shape):
        model = Sequential()
        model.add(LSTM(6, input_shape=input_shape, return_sequences=True))
        model.add(LSTM(150, return_sequences=True))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        self.model = model

    def train(self, epochs=100, batch_size=60):
        train_X, train_y, test_X, test_y = self.load_and_prepare_data()
        self.build_model((train_X.shape[1], train_X.shape[2]))
        history = self.model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_X, test_y), verbose=2, shuffle=False)
        return history

    def save(self, model_path):
        self.model.save(model_path)

    def load(self, model_path):
        self.model = load_model(model_path)

    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat



2025-07-10 09:56:40.991989: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 09:56:41.011312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752112601.030694   53524 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752112601.037015   53524 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752112601.052558   53524 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 新币汇率预测

In [ ]:
import numpy as np
import pandas as pd
import joblib


def predict(name:str):
    #CNY_HKD_to_exchange_rate.csv
    model = CurrencyLSTMModel(currency_name=name, data_path=f"CNY_{name}_to_exchange_rate.csv", look_back=100)
    model.train()
    model.save(f"{name}_model.keras")
    model.load(f"{name}_model.keras")
    joblib.dump(model.scaler, 'scaler.save')
    model.scaler = joblib.load('scaler.save')
    csvname = f"CNY_{name}_to_exchange_rate.csv"
    # 读取原始数据
    dataset = pd.read_csv(csvname, header=0, index_col=0)
    values = dataset.values.astype('float32')


    # 归一化
    scaled = model.scaler.transform(values)

    # 取最近100天
    last_100 = scaled[-100:]  # shape: (100, 1)

    # 递归预测未来20天
    future_steps = 20
    future_preds = []

    input_seq = last_100.reshape((1, 100, 1))

    for _ in range(future_steps):
        yhat = model.predict(input_seq)
        future_preds.append(yhat[0, 0])
        # 更新输入序列：去掉最早一天，加上最新预测
        new_input = np.append(input_seq[0, 1:, 0], yhat[0, 0])
        input_seq = new_input.reshape((1, 100, 1))

    # 反归一化
    future_preds = np.array(future_preds).reshape(-1, 1)
    future_real = model.scaler.inverse_transform(future_preds)
    print("未来20天预测汇率：")
    print(future_real.flatten())

targets = ["JPY", "KRW", "SGD", "THB", "HKD", "MYR"]  # 多个目标货币
for target in targets:
    print(f"预测 {target} 对人民币的汇率：")
    predict(target)
    print("-" * 30)

预测 JPY 对人民币的汇率：
Epoch 1/100


/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 5s - 3s/step - loss: 0.3330 - val_loss: 0.3181
Epoch 2/100
2/2 - 1s - 284ms/step - loss: 0.2286 - val_loss: 0.1991
Epoch 3/100
2/2 - 1s - 258ms/step - loss: 0.1030 - val_loss: 0.2956
Epoch 4/100
2/2 - 0s - 229ms/step - loss: 0.2253 - val_loss: 0.1485
Epoch 5/100
2/2 - 0s - 225ms/step - loss: 0.1349 - val_loss: 0.1730
Epoch 6/100
2/2 - 0s - 234ms/step - loss: 0.1148 - val_loss: 0.1957
Epoch 7/100
2/2 - 0s - 243ms/step - loss: 0.1214 - val_loss: 0.1988
Epoch 8/100
2/2 - 1s - 251ms/step - loss: 0.1183 - val_loss: 0.1876
Epoch 9/100
2/2 - 1s - 255ms/step - loss: 0.1069 - val_loss: 0.1748
Epoch 10/100
2/2 - 1s - 251ms/step - loss: 0.1036 - val_loss: 0.1631
Epoch 11/100
2/2 - 0s - 247ms/step - loss: 0.1041 - val_loss: 0.1537
Epoch 12/100
2/2 - 0s - 233ms/step - loss: 0.1063 - val_loss: 0.1481
Epoch 13/100
2/2 - 0s - 237ms/step - loss: 0.1083 - val_loss: 0.1471
Epoch 14/100
2/2 - 0s - 220ms/step - loss: 0.1084 - val_loss: 0.1495
Epoch 15/100
2/2 - 1s - 252ms/step - loss: 0.1069 - val_lo

/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 4s - 2s/step - loss: 0.4439 - val_loss: 0.5372
Epoch 2/100
2/2 - 1s - 251ms/step - loss: 0.2651 - val_loss: 0.1462
Epoch 3/100
2/2 - 1s - 251ms/step - loss: 0.1027 - val_loss: 0.3719
Epoch 4/100
2/2 - 1s - 292ms/step - loss: 0.2250 - val_loss: 0.1286
Epoch 5/100
2/2 - 1s - 283ms/step - loss: 0.1311 - val_loss: 0.1304
Epoch 6/100
2/2 - 1s - 252ms/step - loss: 0.1473 - val_loss: 0.2054
Epoch 7/100
2/2 - 0s - 234ms/step - loss: 0.1541 - val_loss: 0.1846
Epoch 8/100
2/2 - 1s - 277ms/step - loss: 0.1343 - val_loss: 0.1121
Epoch 9/100
2/2 - 0s - 248ms/step - loss: 0.1075 - val_loss: 0.0698
Epoch 10/100
2/2 - 1s - 253ms/step - loss: 0.1055 - val_loss: 0.1238
Epoch 11/100
2/2 - 0s - 229ms/step - loss: 0.1221 - val_loss: 0.1172
Epoch 12/100
2/2 - 1s - 268ms/step - loss: 0.1112 - val_loss: 0.0699
Epoch 13/100
2/2 - 0s - 244ms/step - loss: 0.1048 - val_loss: 0.0732
Epoch 14/100
2/2 - 0s - 233ms/step - loss: 0.1051 - val_loss: 0.0819
Epoch 15/100
2/2 - 0s - 220ms/step - loss: 0.1043 - val_lo

/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
